In [13]:
import os
os.remove("./densenet/densenet_stage2_all-0.663.hdf5")

In [ ]:
!unzip data_augmentation_1

### Importing Libraries

In [14]:
#Importing required libraries
import os
import json
import urllib
import numpy as np 
import pandas as pd
import seaborn as sns 
import matplotlib.pyplot as plt
%matplotlib inline
import random as rn
import pickle
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.calibration import CalibratedClassifierCV
from sklearn.metrics import log_loss
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
import datetime
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras import optimizers
from tensorflow.keras import applications
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.callbacks import ModelCheckpoint, History
from tensorflow.keras.layers import Dense, Activation, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D,Input
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img,img_to_array
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping

In [15]:
#Creating a folder to save models
os.mkdir('densenet')

In [28]:
#Creating a funciton which sets layers to non trainable if required
def non_trainable(model):
    for i in range(len(model.layers)):
      model.layers[i].trainable = False
    return model

In [29]:
#Plotting metrics for neural networks models
def plot_metrics(nn_model,metric):
  plt.figure(figsize = (10,5))
  plt.plot(nn_model.history.history[metric])
  plt.plot(nn_model.history.history['val_'+metric])
  plt.title('model '+metric)
  plt.ylabel(metric)
  plt.xlabel('epoch')
  plt.legend(['train', 'test'], loc='upper left')
  plt.show()
  idx = np.argmax(nn_model.history.history['val_accuracy'])
  print('The '+metric+' for final model is ',nn_model.history.history['val_'+metric][idx])

In [30]:
#Plotting confusion matrix
def binary_confusion_matrix(true_y,pred_y,labels_list):
  Cm = confusion_matrix(true_y,pred_y)
  print("-"*50, "Confusion matrix", "-"*50)
  cmap=sns.light_palette("blue")
  plt.figure(figsize=(10,5))
  sns.heatmap(Cm, annot=True, cmap=cmap, fmt=".3f", xticklabels=labels_list, yticklabels=labels_list)
  plt.xlabel('Predicted Class')
  plt.ylabel('Original Class')
  plt.show()

In [31]:
def multiclass_confusion_matrix(true_y,pred_y,labels_list):
  Cm = confusion_matrix(true_y,pred_y)
  Pm = (Cm/Cm.sum(axis = 0))
  Rm = (((Cm.T)/(Cm.sum(axis =1))).T)
  cmap = sns.light_palette('blue')
  mt = [Cm,Pm,Rm]
  names = ['Confusion Matrix','Precision Matrix','Recall Matrix']
  print('Average Precision is',np.trace(Pm)/3)
  print('Average Recall is',np.trace(Rm)/3)
  for i in range(len(mt)):
    print('-'*50,names[i],'-'*50)
    plt.figure(figsize = (10,5))
    sns.heatmap(mt[i],annot = True,cmap = cmap,fmt = ".3f",xticklabels = labels_list,yticklabels = labels_list)
    plt.xlabel('Predicted Class')
    plt.ylabel('Original Class')
    plt.show()

In [32]:
def compute_precision_recall(true_y,true_pred):
  tp,fn,fp,tn = confusion_matrix(true_y,true_pred).ravel()
  precision = tp/(tp+fp)
  recall = tp/(tp+fn)
  return precision,recall

In [33]:
densenet_scores = pd.DataFrame(columns = ['Model','Accuracy','Precision','Recall'])

# Orginal Data

In [34]:
#Data for Neural Networks
input_shapes = (256,256,3)
batch_size = 8

In [35]:
#Classes
stage1_class_labels = ['damaged','not_damaged']
stage2_class_labels = ['front','rear','side']
stage3_class_labels = ['minor','moderate','severe']

## Densenet

### Baseline Model

In [36]:
def model_features(model,train_dir,test_dir):
    train_gen = ImageDataGenerator()
    test_gen = ImageDataGenerator()
    
    train_fd = train_gen.flow_from_directory(train_dir,target_size = (256,256),batch_size = 1,shuffle = False)
    test_fd = test_gen.flow_from_directory(test_dir,target_size = (256,256),batch_size = 1,shuffle = False)
    
    train_features = model.predict(train_fd)
    test_features = model.predict(test_fd)
    
    return train_features,test_features,train_fd,test_fd

In [37]:
def target_feat(fd_class):
    temp_dict = dict(fd_class.class_indices)
    keys = list(temp_dict.keys())
    values = list(temp_dict.values())
    
    y = [keys[values.index(i)] for i in fd_class.classes]
    
    return y

In [38]:
#Importing Densenet model and setting layers as non trainable
densenet = applications.DenseNet201(include_top=False,weights = 'imagenet',input_shape=(256,256,3))
densenet = non_trainable(densenet)
fc = Flatten()(densenet.output)
model_densenet = Model(inputs = densenet.input,outputs = fc)

### Stage 1

In [39]:
train_data_dir = '../input/jjjjjjjj/content/data_augmentation_1/data_1/train'
test_data_dir = '../input/jjjjjjjj/content/data_augmentation_1/data_1/test'

In [40]:
train_feat,test_feat,train_fd,test_fd = model_features(model_densenet,train_data_dir, test_data_dir)

In [41]:
train_df = pd.DataFrame(train_feat)
x_test = pd.DataFrame(test_feat)

In [42]:
train_class = target_feat(train_fd)
y_test = target_feat(test_fd)

In [43]:
#Dividing the datasets into train and CV for hyperparameter tuning.
x_train,x_cv,y_train,y_cv = train_test_split(train_df,train_class,stratify=train_class,test_size=0.20)

In [ ]:
#Hyperparameter tuning
alpha = [10 ** x for x in range(-4, 3)]
cv_log_error_array=[] 
cv_acc_array = []
for i in tqdm(alpha):
    logisticR=LogisticRegression(penalty='l2',C=i,class_weight= 'balanced')
    logisticR.fit(x_train,y_train)
    cv_acc_array.append(accuracy_score(y_cv,logisticR.predict(x_cv)))
    
for i in range(len(cv_acc_array)):
    print('Accuracy for alpha '+str(alpha[i])+' is ',cv_acc_array[i])

best_alpha = np.argmax(cv_acc_array)
fig, ax = plt.subplots(figsize=(10,10))
ax.plot(alpha, cv_acc_array,c='g')
for i, txt in enumerate(np.round(cv_acc_array,3)):
    ax.annotate((alpha[i],np.round(txt,3)), (alpha[i],cv_acc_array[i]))
plt.grid()
plt.title("Cross Validation accuracy for each alpha")
plt.xlabel("Alpha i's")
plt.ylabel("Accuracy measure")
plt.show()

In [ ]:
#Final model using best paramaters from hyperparameter tuning.
base_final=LogisticRegression(penalty='l2',C=alpha[best_alpha])
base_final.fit(x_train,y_train)

#saving the model
save_path = 'densenet/densenet_stage1_baseline.sav'
pickle.dump(base_final,open(save_path,'wb'))

#Predcitions and visualization of Confusion matrix
predict_y = base_final.predict(x_test)
print ('Accuracy on test data for final baseline model',accuracy_score(y_test, predict_y))
precision,recall = compute_precision_recall(y_test,predict_y)
print ('Precision on test data for final baseline model',precision)
print ('Recall on test data for final baseline model',recall)
binary_confusion_matrix(y_test,predict_y,stage1_class_labels)

In [ ]:
densenet_scores = densenet_scores.append({'Model':'Densenet_stage1_baseline','Accuracy':'0.870','Precision':'0.873','Recall':'0.865'},ignore_index= True)

### Stage 2

In [ ]:
train_data_dir = '../input/jjjjjjjj/content/data_augmentation_1/data_2/train'
test_data_dir = '../input/jjjjjjjj/content/data_augmentation_1/data_2/test'

In [ ]:
train_feat,test_feat,train_fd,test_fd = model_features(model_densenet,train_data_dir, test_data_dir)

In [ ]:
train_df = pd.DataFrame(train_feat)
x_test = pd.DataFrame(test_feat)

In [ ]:
train_class = target_feat(train_fd)
y_test = target_feat(test_fd)

In [ ]:
#Dividing the datasets into train and CV for hyperparameter tuning.
x_train,x_cv,y_train,y_cv = train_test_split(train_df,train_class,stratify=train_class,test_size=0.20)

In [ ]:
#Hyperparameter tuning
alpha = [10 ** x for x in range(-4, 3)]
cv_log_error_array=[] 
cv_acc_array = []
for i in tqdm(alpha):
    logisticR=LogisticRegression(penalty='l2',C=i,class_weight= 'balanced')
    logisticR.fit(x_train,y_train)
    cv_acc_array.append(accuracy_score(y_cv,logisticR.predict(x_cv)))
    
for i in range(len(cv_acc_array)):
    print('Accuracy for alpha '+str(alpha[i])+' is ',cv_acc_array[i])

best_alpha = np.argmax(cv_acc_array)
fig, ax = plt.subplots(figsize=(10,10))
ax.plot(alpha, cv_acc_array,c='g')
for i, txt in enumerate(np.round(cv_acc_array,3)):
    ax.annotate((alpha[i],np.round(txt,3)), (alpha[i],cv_acc_array[i]))
plt.grid()
plt.title("Cross Validation accuracy for each alpha")
plt.xlabel("Alpha i's")
plt.ylabel("Accuracy measure")
plt.show()

In [ ]:
#Final model using best paramaters from hyperparameter tuning.
base_final=LogisticRegression(penalty='l2',C=alpha[best_alpha])
base_final.fit(x_train,y_train)

#saving the model 
save_path = 'densenet/densenet_stage2_baseline.sav'
pickle.dump(base_final,open(save_path,'wb'))

#Predcitions and visualization of Confusion matrix
predict_y = base_final.predict(x_test)
print ('Accuracy on test data for final baseline model',accuracy_score(y_test, predict_y))
multiclass_confusion_matrix(y_test,predict_y,stage2_class_labels)

In [ ]:
densenet_scores = densenet_scores.append({'Model':'Densenet_stage2_baseline','Accuracy':'0.536','Precision':'0.535','Recall':'0.532'},ignore_index= True)

### Stage 3

In [ ]:
train_data_dir = '../input/jjjjjjjj/content/data_augmentation_1/data_3/train'
test_data_dir = '../input/jjjjjjjj/content/data_augmentation_1/data_3/test'

In [ ]:
train_feat,test_feat,train_fd,test_fd = model_features(model_densenet,train_data_dir, test_data_dir)

In [ ]:
train_df = pd.DataFrame(train_feat)
x_test = pd.DataFrame(test_feat)

In [ ]:
train_class = target_feat(train_fd)
y_test = target_feat(test_fd)

In [ ]:
#Dividing the datasets into train and CV for hyperparameter tuning.
x_train,x_cv,y_train,y_cv = train_test_split(train_df,train_class,stratify=train_class,test_size=0.20)

In [ ]:
#Hyperparameter tuning
alpha = [10 ** x for x in range(-4, 3)]
cv_log_error_array=[] 
cv_acc_array = []
for i in tqdm(alpha):
    logisticR=LogisticRegression(penalty='l2',C=i,class_weight= 'balanced')
    logisticR.fit(x_train,y_train)
    cv_acc_array.append(accuracy_score(y_cv,logisticR.predict(x_cv)))
    
for i in range(len(cv_acc_array)):
    print('Accuracy for alpha '+str(alpha[i])+' is ',cv_acc_array[i])

best_alpha = np.argmax(cv_acc_array)
fig, ax = plt.subplots(figsize=(10,10))
ax.plot(alpha, cv_acc_array,c='g')
for i, txt in enumerate(np.round(cv_acc_array,3)):
    ax.annotate((alpha[i],np.round(txt,3)), (alpha[i],cv_acc_array[i]))
plt.grid()
plt.title("Cross Validation accuracy for each alpha")
plt.xlabel("Alpha i's")
plt.ylabel("Accuracy measure")
plt.show()

In [ ]:
#Final model using best paramaters from hyperparameter tuning.
base_final=LogisticRegression(penalty='l2',C=alpha[best_alpha])
base_final.fit(x_train,y_train)

#saving the model
save_path = 'densenet/densenet_stage3_baseline.sav'
pickle.dump(base_final,open(save_path,'wb'))

#Predcitions and visualization of Confusion matrix
predict_y = base_final.predict(x_test)
print ('Accuracy on test data for final baseline model',accuracy_score(y_test, predict_y))
multiclass_confusion_matrix(y_test,predict_y,stage3_class_labels)

In [ ]:
densenet_scores = densenet_scores.append({'Model':'Densenet_stage3_baseline','Accuracy':'0.485','Precision':'0.480','Recall':'0.475'},ignore_index= True)

### CNN (Training FC layers only)

In [ ]:
def create_model(n_classes,output_activation):
    os.environ['PYTHONHASHSEED'] = '0'
    tf.keras.backend.clear_session()

    ## Set the random seed values to regenerate the model.
    np.random.seed(0)
    rn.seed(0)

    #Input layer
    input_layer = Input(shape=(256,256,3),name='Input_Layer')

    #Adding pretrained model
    densenet = applications.DenseNet201(include_top=False,weights = 'imagenet',input_tensor = input_layer)
    densenet = non_trainable(densenet)

    #Flatten
    flatten = Flatten(data_format='channels_last',name='Flatten')(densenet.output)

    #FC layer
    FC1 = Dense(units=512,activation='relu',name='FC1')(flatten)

    #FC layer
    FC2 = Dense(units=256,activation='relu',name='FC2')(FC1)

    #Dropout layer
    droput1 = Dropout(0.5)(FC2)

    #output layer
    Out = Dense(units=n_classes,activation=output_activation,name='Output')(droput1)

    #Creating the Model
    model = Model(inputs=input_layer,outputs=Out)

    return model

### Stage 1

In [ ]:
model = create_model(1,'sigmoid')

In [ ]:
#Compiling the model
model.compile(loss = 'binary_crossentropy', optimizer = optimizers.SGD(learning_rate=0.00001, momentum=0.9), metrics=["accuracy"])

In [ ]:
model.summary()

In [ ]:
#Model saving based on validation accuracy score
filepath="densenet/densenet_stage1_fc-{val_accuracy:.3f}.hdf5"
checkpoint = ModelCheckpoint(filepath=filepath, monitor='val_accuracy',  verbose=1, save_best_only=True, mode='auto')

In [ ]:
traindatagen = ImageDataGenerator()
testdatagen = ImageDataGenerator()

In [ ]:
train_data_dir = '../input/jjjjjjjj/content/data_augmentation_1/data_1/train'
test_data_dir = '../input/jjjjjjjj/content/data_augmentation_1/data_1/test'

train_generator = traindatagen.flow_from_directory(train_data_dir,target_size = (256,256),batch_size = batch_size, class_mode = "binary")
test_generator = testdatagen.flow_from_directory(test_data_dir,target_size = (256,256),batch_size = batch_size, class_mode = "binary")

n_validation_steps = 460/batch_size
n_steps_epoch = 3680/batch_size

In [ ]:
model.fit(train_generator ,validation_data = test_generator,validation_steps = n_validation_steps,steps_per_epoch=n_steps_epoch,epochs=50,callbacks=[checkpoint])

In [ ]:
for i in ['loss','accuracy']:
  plot_metrics(model,i)

In [47]:
x_test = testdatagen.flow_from_directory(test_data_dir,target_size = (256,256),batch_size = 1, class_mode = "binary",shuffle = False)

In [48]:
best_model = load_model('densenet/densenet_stage1_fc-0.883.hdf5')

In [49]:
y_pred = best_model.predict(x_test)

In [50]:
y_true = target_feat(x_test)
y_predicted = []
for i in y_pred:
  if i <= .5:
    y_predicted.append('damaged')
  elif i > .5:
    y_predicted.append('not_damaged')

In [51]:
precision,recall = compute_precision_recall(y_true,y_predicted)
print ('Precision on test data for final baseline model',precision)
print ('Recall on test data for final baseline model',recall)
binary_confusion_matrix(y_true,y_predicted,stage1_class_labels)

In [52]:
densenet_scores = densenet_scores.append({'Model':'Densenet_stage1 FC','Accuracy':'0.883','Precision':'0.888','Recall':'0.900'},ignore_index= True)

### Stage 2

In [53]:
model = create_model(3,'softmax')

In [54]:
#Compiling the model
model.compile(loss = 'categorical_crossentropy', optimizer = optimizers.SGD(learning_rate=0.00001, momentum=0.9), metrics=["accuracy"])

In [55]:
model.summary()

In [56]:
#Model saving based on validation accuracy score
filepath="densenet/densenet_stage2_fc-{val_accuracy:.3f}.hdf5"
checkpoint = ModelCheckpoint(filepath=filepath, monitor='val_accuracy',verbose=1, save_best_only=True, mode='auto')

In [57]:
traindatagen = ImageDataGenerator()
testdatagen = ImageDataGenerator()

In [58]:
train_data_dir = '../input/jjjjjjjj/content/data_augmentation_1/data_2/train'
test_data_dir = '../input/jjjjjjjj/content/data_augmentation_1/data_2/test'

train_generator = traindatagen.flow_from_directory(train_data_dir,target_size = (256,256),batch_size = batch_size)
test_generator = testdatagen.flow_from_directory(test_data_dir,target_size = (256,256),batch_size = batch_size)

n_validation_steps = 179/batch_size
n_steps_epoch = 1970/batch_size

In [59]:
model.fit(train_generator ,validation_data = test_generator,validation_steps = n_validation_steps,steps_per_epoch=n_steps_epoch,epochs=50,callbacks=[checkpoint])

In [62]:
for i in ['loss','accuracy']:
  plot_metrics(model,i)

In [60]:
x_test = testdatagen.flow_from_directory(test_data_dir,target_size = (256,256),batch_size = 1,shuffle = False)

In [61]:
best_model = load_model('densenet/densenet_stage2_fc-0.587.hdf5')

In [63]:
y_pred = best_model.predict(x_test)

In [64]:
y_true = target_feat(x_test)
y_predicted = []
for i in y_pred:
  n = np.argmax(i)
  if n == 0:
    y_predicted.append('front')
  elif n== 1:
    y_predicted.append('rear')
  elif n == 2:
    y_predicted.append('side')

In [65]:
multiclass_confusion_matrix(y_true,y_predicted,stage2_class_labels)

In [66]:
densenet_scores = densenet_scores.append({'Model':'Densenet_stage2 FC','Accuracy':'0.587','Precision':'0.588','Recall':'0.574'},ignore_index= True)

### Stage 3

In [67]:
model = create_model(3,'softmax')

In [68]:
#Compiling the model
model.compile(loss = 'categorical_crossentropy', optimizer = optimizers.SGD(lr=0.00001, momentum=0.9), metrics=["accuracy"])

In [69]:
model.summary()

In [70]:
#Model saving based on validation accuracy score
filepath="densenet/densenet_stage3_fc-{val_accuracy:.3f}.hdf5"
checkpoint = ModelCheckpoint(filepath=filepath, monitor='val_accuracy',verbose=1, save_best_only=True, mode='auto')

In [71]:
traindatagen = ImageDataGenerator()
testdatagen = ImageDataGenerator()

In [72]:
train_data_dir = '../input/jjjjjjjj/content/data_augmentation_1/data_3/train'
test_data_dir = '../input/jjjjjjjj/content/data_augmentation_1/data_3/test'

train_generator = traindatagen.flow_from_directory(train_data_dir,target_size = (256,256),batch_size = batch_size)
test_generator = testdatagen.flow_from_directory(test_data_dir,target_size = (256,256),batch_size = batch_size)

n_validation_steps = 171/batch_size
n_steps_epoch = 1958/batch_size

In [73]:
model.fit(train_generator ,validation_data = test_generator,validation_steps = n_validation_steps,steps_per_epoch=n_steps_epoch,epochs=50,callbacks=[checkpoint])

In [74]:
for i in ['loss','accuracy']:
  plot_metrics(model,i)

In [75]:
x_test = testdatagen.flow_from_directory(test_data_dir,target_size = (256,256),batch_size = 1,shuffle = False)

In [76]:
best_model = load_model('densenet/densenet_stage3_fc-0.550.hdf5')

In [77]:
y_pred = best_model.predict(x_test)

In [78]:
y_true = target_feat(x_test)
y_predicted = []
for i in y_pred:
  n = np.argmax(i)
  if n == 0:
    y_predicted.append('minor')
  elif n== 1:
    y_predicted.append('moderate')
  elif n == 2:
    y_predicted.append('severe')

In [79]:
multiclass_confusion_matrix(y_true,y_predicted,stage3_class_labels)

In [81]:
densenet_scores = densenet_scores.append({'Model':'Densenet_stage3 FC','Accuracy':'0.550','Precision':'0.546','Recall':'0.530'},ignore_index= True)

### CNN (Training All layers only)

In [4]:
def create_model(n_classes,output_activation):
    os.environ['PYTHONHASHSEED'] = '0'
    tf.keras.backend.clear_session()

    ## Set the random seed values to regenerate the model.
    np.random.seed(0)
    rn.seed(0)

    #Input layer
    input_layer = Input(shape=(256,256,3),name='Input_Layer')

    #Adding pretrained model
    densenet = applications.DenseNet201(include_top=False,weights = 'imagenet',input_tensor = input_layer)

    #Flatten
    flatten = Flatten(data_format='channels_last',name='Flatten')(densenet.output)

    #FC layer
    FC1 = Dense(units=512,activation='relu',name='FC1')(flatten)

    #FC layer
    FC2 = Dense(units=256,activation='relu',name='FC2')(FC1)

    #Dropout layer
    droput1 = Dropout(0.5)(FC2)

    #output layer
    Out = Dense(units=n_classes,activation=output_activation,name='Output')(droput1)

    #Creating the Model
    model = Model(inputs=input_layer,outputs=Out)

    return model

### Stage 1

In [5]:
model = create_model(1,'sigmoid')

In [115]:
#Compiling the model
model.compile(loss = 'binary_crossentropy', optimizer = optimizers.SGD(learning_rate=0.00001, momentum=0.9), metrics=["accuracy"])

In [116]:
model.summary()

In [125]:
os.remove("./densenet/densenet_stage2_all-0.659.hdf5")

In [1]:
#Model saving based on validation accuracy score
filepath="densenet/densenet_stage1_all-{val_accuracy:.3f}.hdf5"
checkpoint = ModelCheckpoint(filepath=filepath, monitor='val_accuracy',  verbose=1, save_best_only=True, mode='auto')

In [2]:
traindatagen = ImageDataGenerator()
testdatagen = ImageDataGenerator()

In [129]:
train_data_dir = '../input/jjjjjjjj/content/data_augmentation_1/data_1/train'
test_data_dir = '../input/jjjjjjjj/content/data_augmentation_1/data_1/test'

train_generator = traindatagen.flow_from_directory(train_data_dir,target_size = (256,256),batch_size = batch_size, class_mode = "binary")
test_generator = testdatagen.flow_from_directory(test_data_dir,target_size = (256,256),batch_size = batch_size, class_mode = "binary")

n_validation_steps = 460/batch_size
n_steps_epoch = 3680/batch_size

In [ ]:
model.fit(train_generator ,validation_data = test_generator,validation_steps = n_validation_steps,steps_per_epoch=n_steps_epoch,epochs=50,callbacks=[checkpoint])

In [96]:
for i in ['loss','accuracy']:
  plot_metrics(model,i)

In [97]:
x_test = testdatagen.flow_from_directory(test_data_dir,target_size = (256,256),batch_size = 1, class_mode = "binary",shuffle = False)

In [112]:
best_model = load_model('densenet/densenet_stage1_all-0.952.hdf5')

In [99]:
y_pred = best_model.predict(x_test)

In [100]:
y_true = target_feat(x_test)
y_predicted = []
for i in y_pred:
  if i <= .5:
    y_predicted.append('damaged')
  elif i > .5:
    y_predicted.append('not_damaged')

In [101]:
precision,recall = compute_precision_recall(y_true,y_predicted)
print ('Precision on test data for final baseline model',precision)
print ('Recall on test data for final baseline model',recall)
binary_confusion_matrix(y_true,y_predicted,stage1_class_labels)

In [102]:
densenet_scores = densenet_scores.append({'Model':'Densenet_stage1 all','Accuracy':'0.959','Precision':'0.949','Recall':'0.970'},ignore_index= True)

### Stage 2

In [103]:
model = create_model(3,'softmax')

In [104]:
#Compiling the model
model.compile(loss = 'categorical_crossentropy', optimizer = optimizers.SGD(lr=0.00001, momentum=0.9), metrics=["accuracy"])

In [105]:
model.summary()

In [86]:
#Model saving based on validation accuracy score
filepath="densenet/densenet_stage2_all-{val_accuracy:.3f}.hdf5"
checkpoint = ModelCheckpoint(filepath=filepath, monitor='val_accuracy',verbose=1, save_best_only=True, mode='auto')

In [87]:
traindatagen = ImageDataGenerator()
testdatagen = ImageDataGenerator()

In [88]:
train_data_dir = '../input/jjjjjjjj/content/data_augmentation_1/data_2/train'
test_data_dir = '../input/jjjjjjjj/content/data_augmentation_1/data_2/test'

train_generator = traindatagen.flow_from_directory(train_data_dir,target_size = (256,256),batch_size = batch_size)
test_generator = testdatagen.flow_from_directory(test_data_dir,target_size = (256,256),batch_size = batch_size)

n_validation_steps = 179/batch_size
n_steps_epoch = 1970/batch_size

In [89]:
model.fit(train_generator ,validation_data = test_generator,validation_steps = n_validation_steps,steps_per_epoch=n_steps_epoch,epochs=50,callbacks=[checkpoint])

In [90]:
for i in ['loss','accuracy']:
  plot_metrics(model,i)

In [91]:
x_test = testdatagen.flow_from_directory(test_data_dir,target_size = (256,256),batch_size = 1,shuffle = False)

In [92]:
best_model = load_model('densenet/densenet_stage2_all-0.667.hdf5')

In [93]:
y_pred = best_model.predict(x_test)

In [94]:
y_true = target_feat(x_test)
y_predicted = []
for i in y_pred:
  n = np.argmax(i)
  if n == 0:
    y_predicted.append('front')
  elif n== 1:
    y_predicted.append('rear')
  elif n == 2:
    y_predicted.append('side')

In [95]:
multiclass_confusion_matrix(y_true,y_predicted,stage2_class_labels)

In [96]:
densenet_scores = densenet_scores.append({'Model':'Densenet_stage2 all','Accuracy':'0.804','Precision':'0.807','Recall':'0.789'},ignore_index= True)

### Stage 3

In [97]:
model = create_model(3,'softmax')

In [98]:
#Compiling the model
model.compile(loss = 'categorical_crossentropy', optimizer = optimizers.SGD(lr=0.00001, momentum=0.9), metrics=["accuracy"])

In [99]:
model.summary()

In [ ]:
tf.keras.utils.plot_model(
    model, to_file='model.png', show_shapes=False, show_layer_names=True,rankdir='TB', expand_nested=False, dpi=96)

In [100]:
#Model saving based on validation accuracy score
filepath="densenet/densenet_stage3_all-{val_accuracy:.3f}.hdf5"
checkpoint = ModelCheckpoint(filepath=filepath, monitor='val_accuracy',verbose=1, save_best_only=True, mode='auto')

In [101]:
traindatagen = ImageDataGenerator()
testdatagen = ImageDataGenerator()

In [102]:
train_data_dir = '../input/jjjjjjjj/content/data_augmentation_1/data_3/train'
test_data_dir = '../input/jjjjjjjj/content/data_augmentation_1/data_3/test'

train_generator = traindatagen.flow_from_directory(train_data_dir,target_size = (256,256),batch_size = batch_size)
test_generator = testdatagen.flow_from_directory(test_data_dir,target_size = (256,256),batch_size = batch_size)

n_validation_steps = 171/batch_size
n_steps_epoch = 1958/batch_size

In [103]:
model.fit(train_generator ,validation_data = test_generator,validation_steps = n_validation_steps,steps_per_epoch=n_steps_epoch,epochs=50,callbacks=[checkpoint])

In [104]:
for i in ['loss','accuracy']:
  plot_metrics(model,i)

In [105]:
x_test = testdatagen.flow_from_directory(test_data_dir,target_size = (256,256),batch_size = 1,shuffle = False)

In [106]:
best_model = load_model('densenet/densenet_stage3_all-0.673.hdf5')

In [107]:
y_pred = best_model.predict(x_test)

In [108]:
y_true = target_feat(x_test)
y_predicted = []
for i in y_pred:
  n = np.argmax(i)
  if n == 0:
    y_predicted.append('minor')
  elif n== 1:
    y_predicted.append('moderate')
  elif n == 2:
    y_predicted.append('severe')

In [109]:
multiclass_confusion_matrix(y_true,y_predicted,stage3_class_labels)

In [110]:
densenet_scores = densenet_scores.append({'Model':'Densenet_stage3 all','Accuracy':'0.673','Precision':'0.667','Recall':'0.671'},ignore_index= True)

In [111]:
densenet_scores.to_csv('densenet_scores.csv',index=False)